Оптимизация гиперпараметров 
     1. С помощью [optuna]() взять пример, аналогичный третьему туториалу документации, используя sklearn и с другим датасетом, выбрать другие  алгоритмы классификации и клстеризации не из туториала  и визуализировать графики для полученного процесса
        1. В качестве других моделей подойдут любые алгоритмы классификации и регрессии из sklearn которые не использовались в туториале
     2. Использовать 2 разных семплера и прунера
     3. При процессе оптимизации гиперпараметров использовать общую память через postgreSQL
     4. В качестве отчёта выступают: исходный код, инструкция запуска реляционной БД. 

In [1]:
import logging
import sys

import optuna
import sklearn.datasets
import sklearn.model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

In [39]:
def objective(trial):
    wine = sklearn.datasets.load_wine()
    X, y = wine.data, wine.target
    X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(
        X, y, test_size=0.25, random_state=0
    )
    classes = list(set(y))
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    model_name = trial.suggest_categorical("model", ["MultinomialNB", "MLPClassifier"])

    if model_name == "MultinomialNB":
        alpha = trial.suggest_float("alpha", 1e-5, 1e1, log=True)
        clf = MultinomialNB(alpha=alpha)


        for step in range(100): 
            clf.partial_fit(X_train, y_train, classes=classes)
            intermediate_value = 1.0 - clf.score(X_valid, y_valid)
            trial.report(intermediate_value, step)
            if trial.should_prune():
                raise optuna.TrialPruned()

    else: 
        hidden_layer_sizes = trial.suggest_int("hidden_layer_sizes", 10, 100, log=True)
        activation = trial.suggest_categorical("activation", ["relu", "tanh", "logistic"])
        solver = trial.suggest_categorical("solver", ["adam", "sgd"])
        alpha_mlp = trial.suggest_float("alpha_mlp", 1e-5, 1e-1, log=True)

        clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, alpha=alpha_mlp, random_state=0, max_iter=500) # Увеличиваем max_iter
        for step in range(100):
            clf.partial_fit(X_train_scaled, y_train, classes=classes)
            intermediate_value = 1.0 - clf.score(X_valid_scaled, y_valid)
            trial.report(intermediate_value, step)
            if trial.should_prune():
                raise optuna.TrialPruned()


    return 1.0 - clf.score(X_valid_scaled if model_name == "MLPClassifier" else X_valid, y_valid)

In [43]:
# Два разных семплера
sampler1 = optuna.samplers.TPESampler()
sampler2 = optuna.samplers.RandomSampler()

# Два разных прунера
pruner1 = optuna.pruners.HyperbandPruner()
pruner2 = optuna.pruners.MedianPruner()

In [2]:
!optuna-dashboard postgresql://aleksandralekseev:@localhost/optuna

Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [23/Dec/2024 15:34:26] "GET / HTTP/1.1" 302 0
127.0.0.1 - - [23/Dec/2024 15:34:26] "GET /dashboard HTTP/1.1" 200 4145
127.0.0.1 - - [23/Dec/2024 15:34:26] "GET /static/bundle.js HTTP/1.1" 200 4158971
127.0.0.1 - - [23/Dec/2024 15:34:27] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 790
127.0.0.1 - - [23/Dec/2024 15:34:27] "GET /apple-touch-icon.png HTTP/1.1" 404 766
127.0.0.1 - - [23/Dec/2024 15:34:27] "GET /favicon.ico HTTP/1.1" 200 7670
127.0.0.1 - - [23/Dec/2024 15:34:27] "GET /api/studies HTTP/1.1" 200 323
127.0.0.1 - - [23/Dec/2024 15:34:29] "GET /api/meta HTTP/1.1" 200 63
/Users/aleksandralekseev/Desktop/karpov_DA/DA_python/venv/lib/python3.9/site-packages/optuna_dashboard/_importance.py:75: ExperimentalWarning: PedAnovaImportanceEvaluator is experimental (supported from v3.6.0). The interface can change in the future.
  return get_param_importances(study, target=target, evaluator=PedAnovaImportanceEval

In [73]:
# Оптимизация с sampler1 и pruner1
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)])
study1 = optuna.create_study(direction="minimize", sampler=sampler1, pruner=pruner1, storage='postgresql://aleksandralekseev:@localhost/optuna')
study1.optimize(objective, n_trials=20)
print(f"Study 1 best params: {study1.best_params}, best value: {study1.best_value}")

[I 2024-12-01 15:24:16,381] A new study created in RDB with name: no-name-2234e946-be70-41b1-8bfc-8448949ab8fb


A new study created in RDB with name: no-name-2234e946-be70-41b1-8bfc-8448949ab8fb


[I 2024-12-01 15:24:17,231] Trial 0 finished with value: 0.5333333333333333 and parameters: {'model': 'MLPClassifier', 'hidden_layer_sizes': 44, 'activation': 'logistic', 'solver': 'sgd', 'alpha_mlp': 3.1979038429721285e-05}. Best is trial 0 with value: 0.5333333333333333.


Trial 0 finished with value: 0.5333333333333333 and parameters: {'model': 'MLPClassifier', 'hidden_layer_sizes': 44, 'activation': 'logistic', 'solver': 'sgd', 'alpha_mlp': 3.1979038429721285e-05}. Best is trial 0 with value: 0.5333333333333333.


[I 2024-12-01 15:24:18,083] Trial 1 finished with value: 0.48888888888888893 and parameters: {'model': 'MLPClassifier', 'hidden_layer_sizes': 16, 'activation': 'logistic', 'solver': 'adam', 'alpha_mlp': 0.0027751476971407006}. Best is trial 1 with value: 0.48888888888888893.


...

Trial 17 finished with value: 0.06666666666666665 and parameters: {'model': 'MultinomialNB', 'alpha': 0.0007967443165527248}. Best is trial 8 with value: 0.0444444444444444.


[I 2024-12-01 15:24:27,124] Trial 18 pruned. 


Trial 18 pruned. 


[I 2024-12-01 15:24:27,185] Trial 19 pruned. 


Trial 19 pruned. 
Study 1 best params: {'model': 'MLPClassifier', 'hidden_layer_sizes': 96, 'activation': 'tanh', 'solver': 'adam', 'alpha_mlp': 0.0016670263901089706}, best value: 0.0444444444444444


In [76]:
# Оптимизация с sampler2 и pruner2
study2 = optuna.create_study(direction="minimize", sampler=sampler2, pruner=pruner2, storage='postgresql://aleksandralekseev:@localhost/optuna')
study2.optimize(objective, n_trials=20)
print(f"Study 2 best params: {study2.best_params}, best value: {study2.best_value}")

[I 2024-12-01 15:30:39,956] A new study created in RDB with name: no-name-e176d08d-c398-4af8-ab4e-1cec404541b0


A new study created in RDB with name: no-name-e176d08d-c398-4af8-ab4e-1cec404541b0


[I 2024-12-01 15:30:40,933] Trial 0 finished with value: 0.06666666666666665 and parameters: {'model': 'MLPClassifier', 'hidden_layer_sizes': 75, 'activation': 'tanh', 'solver': 'adam', 'alpha_mlp': 0.0010173422709758202}. Best is trial 0 with value: 0.06666666666666665.


Trial 0 finished with value: 0.06666666666666665 and parameters: {'model': 'MLPClassifier', 'hidden_layer_sizes': 75, 'activation': 'tanh', 'solver': 'adam', 'alpha_mlp': 0.0010173422709758202}. Best is trial 0 with value: 0.06666666666666665.


[I 2024-12-01 15:30:41,772] Trial 1 finished with value: 0.0888888888888889 and parameters: {'model': 'MLPClassifier', 'hidden_layer_sizes': 82, 'activation': 'tanh', 'solver': 'sgd', 'alpha_mlp': 0.05427293411539595}. Best is trial 0 with value: 0.06666666666666665.


...

Trial 17 pruned. 


[I 2024-12-01 15:30:50,403] Trial 18 finished with value: 0.06666666666666665 and parameters: {'model': 'MultinomialNB', 'alpha': 4.20011053666392e-05}. Best is trial 0 with value: 0.06666666666666665.


Trial 18 finished with value: 0.06666666666666665 and parameters: {'model': 'MultinomialNB', 'alpha': 4.20011053666392e-05}. Best is trial 0 with value: 0.06666666666666665.


[I 2024-12-01 15:30:50,480] Trial 19 pruned. 


Trial 19 pruned. 
Study 2 best params: {'model': 'MLPClassifier', 'hidden_layer_sizes': 75, 'activation': 'tanh', 'solver': 'adam', 'alpha_mlp': 0.0010173422709758202}, best value: 0.06666666666666665
